In [1]:
import numpy as np
import os
import pysynphot as s
import pandas as pd

/usr/local/lib/python2.7/dist-packages/pysynphot/locations.py:44: UserWarning: PYSYN_CDBS is undefined; functionality will be SEVERELY crippled.
  warnings.warn("PYSYN_CDBS is undefined; functionality will be SEVERELY "
/usr/local/lib/python2.7/dist-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '
/usr/local/lib/python2.7/dist-packages/pysynphot/locations.py:155: UserWarning: Extinction files not found in grid/extinction
  warnings.warn('Extinction files not found in %s' % (extdir,))
/usr/local/lib/python2.7/dist-packages/pysynphot/locations.py:133: UserWarning: PYSYN_CDBS is undefined; cannot find mtab/*_tmg.fits file
  warnings.warn("PYSYN_CDBS is undefined; cannot find %s file" % template)
/usr/local/lib/python2.7/dist-packages/pysynphot/locations.py:133: UserWarning: PYSYN_CDBS is undefined; cannot find mtab/*_tmc.fi

In [2]:
jpas_filters_path  = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/JPAS_filters'
jplus_filters_path = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/JPLUS_SDSS_filters'
test_specs         = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/Specs'
results_path       = '/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/Results'
jpas_filters_list  = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/jpas_filters_list.txt', dtype=str)
jplus_filters_list = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Filters/jplus_filters_list.txt', dtype=str)
specs_list         = np.loadtxt('/home/mldantas/Dropbox/DoutoradoIAG/Challenge/Sanity_Check/specslist2.txt', dtype=str)

In [3]:
# Constants --------------------------------------------------------------------------------------------------------
c = 2.99792458E18                  # Light Speed in Angstrom s^-1

In [4]:
# Setting the T80 M1 effective area in cm^2 ------------------------------------------------------------------------
s.setref(area=4400)

In [5]:
# # JPAS -------------------------------------------------------------------------------------------------------------
all_jpas = []
for each_spectrum in specs_list:
    # Transforming the specs from f_lambda to f_nu -----------------------------------------------------------------
    wavelength = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[0])
    f_lambda   = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[1])
    # Simulating the photometry for each J-PAS band ----------------------------------------------------------------
    filter_name = []
    photometry  = []
    for jpas_filters in jpas_filters_list:
        filter_name_i          = jpas_filters.split('.')[0]
        filter_name.append(filter_name_i)
        jpas_filter_bandpass = s.FileBandpass(os.path.join(jpas_filters_path, jpas_filters))        
#         print jpas_filter_bandpass.throughput
        sdss_spectrum        = s.FileSpectrum(os.path.join(test_specs, each_spectrum))
        photometry_i         = s.Observation(sdss_spectrum, jpas_filter_bandpass, binset=np.arange(3000, 11000), 
                                             force='extrap')
        photometry.append(photometry_i.effstim('abmag'))
        print photometry_i.efflam()

    filter_name       = np.array(filter_name)
    photometry        = np.array(photometry)
    photometry_fnu    = 10**(photometry + 48.60)/(-2.5)
    galaxy_simulation_abmag = np.vstack((filter_name, photometry))
    galaxy_simulation_abmag = pd.DataFrame(galaxy_simulation_abmag)
    galaxy_simulation_abmag.to_csv(os.path.join(results_path, os.path.split(each_spectrum)[-1][0:14]+'_JPAS_abmag.csv'), 
                                   sep=',', header=None, index=False)
    galaxy_simulation_fnu = np.vstack((filter_name, photometry_fnu))
    galaxy_simulation_fnu = pd.DataFrame(galaxy_simulation_fnu)
    galaxy_simulation_fnu.to_csv(os.path.join(results_path, os.path.split(each_spectrum)[-1][0:14]+'_JPAS_fnu.csv'), 
                                 sep=',', header=None, index=False)

 # JPLUS ------------------------------------------------------------------------------------------------------------
for each_spectrum in specs_list:
    # Transforming the specs from f_lambda to f_nu -----------------------------------------------------------------
    wavelength = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[0])
    f_lambda   = np.loadtxt(os.path.join(test_specs, each_spectrum), usecols=[1])
    # Simulating the photometry for each J-PAS band ----------------------------------------------------------------
    filter_name2 = []
    photometry2  = []
    for jplus_filters in jplus_filters_list:
        filter_name2_i          = jplus_filters.split('.')[0]
        filter_name2.append(filter_name2_i)
        jplus_filter_bandpass = s.FileBandpass(os.path.join(jplus_filters_path, jplus_filters))        
#         print jplus_filter_bandpass.throughput
        sdss_spectrum        = s.FileSpectrum(os.path.join(test_specs, each_spectrum))
        photometry2_i       = s.Observation(sdss_spectrum, jplus_filter_bandpass, binset=np.arange(3000, 11000), 
                                             force='extrap')
        print photometry_i.efflam()
        photometry2.append(photometry_i.effstim('abmag'))

    filter_name2    = np.array(filter_name2)
    photometry2     = np.array(photometry2)
    photometry2_fnu = 10**(photometry2 + 48.60)/(-2.5)
    galaxy_simulation_abmag = np.vstack((filter_name2, photometry2))
    galaxy_simulation_abmag = pd.DataFrame(galaxy_simulation_abmag)
    galaxy_simulation_abmag.to_csv(os.path.join(results_path, os.path.split(each_spectrum)[-1][0:14]+'_JPLUS_abmag.csv'), 
                                   sep=',', header=None, index=False)
    galaxy_simulation_fnu = np.vstack((filter_name2, photometry2_fnu))
    galaxy_simulation_fnu = pd.DataFrame(galaxy_simulation_fnu)
    galaxy_simulation_fnu.to_csv(os.path.join(results_path, os.path.split(each_spectrum)[-1][0:14]+'_JPLUS_fnu.csv'), 
                                 sep=',', header=None, index=False)

4905.34101521
3837.61010648
4221.75551905
4362.88650308
4279.25783714
4287.87496604
4417.59870952
4517.67232884
4534.27927617
4596.92121962
4690.006436
4805.05057508
4870.59481076
4956.80487682
5081.59692476
5154.67081166
5272.0814665
5346.59497317
5453.34568873
5533.81488158
5634.67686696
5725.92983774
5809.54704369
5917.43295446
6020.86046725
6108.83666686
6197.9379151
6312.64178784
6400.71856214
6501.21109375
6594.68169088
6684.12795388
6786.89700545
6879.85766522
6982.9755472
7074.95149054
7171.64804469
7272.26869865
7371.73215583
7476.00787787
7544.35055137
7672.00974404
7761.68957011
7864.3493576
7955.16919304
7907.19483545
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6338.27249816
6296.16965429
3930.53402901
4793.61939263
5565.1553177
4033.61865011
4054.86321092
4128.91319378
4202.69548747
4327.75863179
4439.413336
4496.44631543
4583.78349722
4687.46615795
4806.70568177
4875.33527225
4

5517.57614837
5615.69092378
5707.88788625
5793.69130871
5904.70002669
6005.65734278
6090.9984143
6181.11352403
6299.02610118
6383.37869678
6485.86772707
6576.42225184
6661.17327807
6764.16804536
6851.45762813
6959.28048358
7050.57618489
7141.15406816
7239.70830165
7339.26510027
7448.65971941
7492.43316285
7641.23150259
7724.30316512
7829.98760901
7921.77040619
8010.92915222
8125.30760058
8181.08786269
7924.85576511
5956.76826699
5956.76826699
5956.76826699
5956.76826699
5956.76826699
5956.76826699
5956.76826699
5956.76826699
6257.57620675
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
3816.31149879
Warning, 9 of 6001 bins contained negative fluxes; they have been set to zero.
3816.31149879
Warning, 9 o